# Volatility Regime Exploratory Validation

This notebook validates the market regime classification logic. We expect higher volatility regimes to align with macroeconomic stress events or trend inflection points.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

# Add root to path to import local modules
sys.path.append(os.path.abspath('..'))

from features.volatility import rolling_volatility
from features.regimes import volatility_regimes

# Load processed 1H data
df = pd.read_csv('../data/processed/XAGUSD_1H.csv', index_index=0, parse_dates=True)

# Calculate Features
df['Volatility'] = rolling_volatility(df)
df['Regime'] = volatility_regimes(df['Volatility'])

print(df['Regime'].value_counts())

In [ ]:
# Visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# Plot Price and Color Background by Regime
ax1.plot(df.index, df['Close'], color='black', alpha=0.7, label='Price')
colors = {'LOW_VOL': 'green', 'MID_VOL': 'yellow', 'HIGH_VOL': 'red'}

for regime in colors:
    mask = df['Regime'] == regime
    ax1.fill_between(df.index, df['Close'].min(), df['Close'].max(), where=mask, color=colors[regime], alpha=0.2)

ax1.set_title('XAGUSD Price Overlaid with Volatility Regimes')
ax1.legend()

# Plot Rolling Volatility
ax2.plot(df.index, df['Volatility'], color='blue', label='30-Period Rolling Vol')
ax2.axhline(df['Volatility'].quantile(0.33), color='gray', linestyle='--')
ax2.axhline(df['Volatility'].quantile(0.66), color='gray', linestyle='--')
ax2.set_title('Rolling Volatility (Log Returns)')
ax2.legend()

plt.tight_layout()
plt.show()